In [3]:
#先引入后面可能用到的包（package）
import pandas as pd  
import numpy as np
import talib as ta
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
%matplotlib inline   
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False


### 股票数据 Stock Data

In [ ]:
# 使用tushare时发现需要国内手机号注册，感觉不用也可以，反正是测试。
# 应该直接使用下载好的股票数据，加工后也能得到ATR

In [ ]:
# 为了实验，使用appl，goog，SPX, NDX, DJI，HSI作为测试数据。可以从我的tradingview下载，
# 截取日线，2016.12.29 - 2022.6.15，

In [48]:
spx = pd.read_csv('/Users/paxton615/Desktop/Stock_Price_csv/turtle_test/SP_SPX, 1D.csv')


In [50]:
# 时间改成dt，只保留年月日
spx.time = pd.to_datetime(spx['time']).dt.date
# 只选部分有用cols，col名全小写
spx = spx[['time', 'open', 'high', 'low', 'close', 'Volume', ]]
spx.columns = ['time', 'open', 'high', 'low', 'close', 'vol' ]


#### 注意：这里的 timeperiod 很可能是未来调参的‘重要参数’

In [54]:
days_high = 20
days_low = 10
days_atr = 20
spx['up']=ta.MAX(spx.high,timeperiod=days_high).shift(1)
#最近N2个交易日最低价
spx['down']=ta.MIN(spx.low,timeperiod=days_low).shift(1)
#每日真实波动幅度
spx['atr']=ta.ATR(spx.high,spx.low,spx.close,timeperiod=days_atr)
spx.tail()


,time,open,high,low,close,vol,up,down,atr
1655,2022-06-13,3838.15,3838.15,3734.30,3749.64,3034000000,4177.51,3900.16,97.191983
1656,2022-06-14,3763.52,3778.18,3705.68,3735.47,2616000000,4177.51,3734.30,95.957384
1657,2022-06-15,3764.05,3837.56,3722.30,3790.00,2730000000,4177.51,3705.68,96.922515
1658,2022-06-16,3728.18,3728.18,3639.77,3666.76,3042000000,4177.51,3705.68,99.587889
1659,2022-06-17,3665.90,3707.71,3636.87,3674.85,4526000000,4177.51,3639.77,98.150495


In [80]:
# 稍微做了点改动在index上，function是可以正常对spx输出结果的，已测试
def my_strategy(data):
    
    x1=data.close>data.up
    x2=data.close.shift(1)<data.up.shift(1)
    x=x1&x2      # 不断更新最高价格
    
    y1=data.close<data.down
    y2=data.close.shift(1)>data.down.shift(1)
    y=y1&y2      # 不断更新最低价格
    
    data.loc[x,'signal']='buy'       # 新建signal col 用于标记’买‘信号
    data.loc[y,'signal']='sell'      # 新建signal col 用于标记’卖‘信号
    
    buy_date=(data[data.signal=='buy'].index) # ’买‘的日期
    sell_date=(data[data.signal=='sell'].index) # ’卖‘的日期
    
    buy_close=data[data.signal=='buy'].close.round(2).tolist()  # ‘买’的close价格
    sell_close=data[data.signal=='sell'].close.round(2).tolist() # ’卖‘的close价格
    return (buy_date,buy_close,sell_date,sell_close)


In [83]:
print(my_strategy(spx))

(Int64Index([  69,   72,   80,   83,   92,   94,  102,  105,  134,  137,
            ...
            1444, 1454, 1456, 1491, 1496, 1500, 1539, 1596, 1598, 1601],
           dtype='int64', length=151), [1951.7, 1978.35, 2022.19, 2027.22, 2063.95, 2072.78, 2082.42, 2094.34, 2099.06, 2105.26, 2109.41, 2129.9, 2163.75, 2173.02, 2182.87, 2190.15, 2163.26, 2187.12, 2198.18, 2213.35, 2241.35, 2271.72, 2298.37, 2307.87, 2365.38, 2369.75, 2395.96, 2388.61, 2399.29, 2415.07, 2430.06, 2453.46, 2459.27, 2473.83, 2488.11, 2500.23, 2519.36, 2555.24, 2561.26, 2575.21, 2581.07, 2591.13, 2599.03, 2602.42, 2627.04, 2647.58, 2675.81, 2695.81, 2767.56, 2810.3, 2872.87, 2723.07, 2746.87, 2772.35, 2782.0, 2793.84, 2798.29, 2809.55, 2820.4, 2850.4, 2874.69, 2914.04, 2930.75, 2616.1, 2681.05, 2724.87, 2744.73, 2775.6, 2792.67, 2822.48, 2854.88, 2867.19, 2873.4, 2892.74, 2907.41, 2933.68, 2939.88, 2917.75, 2954.18, 2964.33, 2995.82, 3013.77, 3019.56, 3025.86, 2976.0, 3000.93, 3022.55, 3066.91, 3120.46, 3133.64

In [74]:
#对K线图和唐奇安通道进行可视化
from pyecharts.charts import Bar, Line, Grid
grid = Grid()
attr=[str(t) for t in spx.time]
#attr=[str(t) for t in spx.time.strftime('%Y%m%d')]
v1=np.array(spx.loc[:,['open','close','low','high']])
v2=np.array(spx.up)
v3=np.array(spx.down)
#kline = Kline("沪深300唐奇安通道",title_text_size=15)
#kline.add("K线图", attr, v1.round(1),is_datazoom_show=True,)

# 成交量
bar= Bar()
bar.add("成交量", attr, spx['vol'],tooltip_tragger="axis", is_legend_show=False, 
        is_yaxis_show=False, yaxis_max=5*max(spx["vol"]))

line = Line()
line.add("上轨线", attr, v2.round(1),is_datazoom_show=True,
         is_smooth=True,is_symbol_show=False,line_width=1.5)
line.add("下轨线", attr, v3.round(1),is_datazoom_show=True,
         is_smooth=True,is_symbol_show=False,line_width=1.5)
#添加买卖信号
bd,bc,sd,sc=my_strategy(spx)
es = EffectScatter("buy")
es.add( "sell", sd, sc, )
es.add("buy", bd, bc,symbol="triangle",)
overlap = Overlap(width=2000, height=600)
overlap.add(kline)
overlap.add(line)
overlap.add(bar,yaxis_index=1, is_add_yaxis=True)
overlap.add(es)
grid.add(overlap, grid_right="10%")
grid

AttributeError: 'Bar' object has no attribute 'add'

In [85]:
!pwd


/Users/paxton615/Github_Personal/Notes/Notes/Turtle_Trading


In [86]:
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker

c = (
    Bar()
    .add_xaxis(Faker.choose())
    .add_yaxis("商家A", Faker.values(), stack="stack1")
    .add_yaxis("商家B", Faker.values(), stack="stack1")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="Bar-堆叠数据（全部）"))
    .render("bar_stack0.html")
)
IFrame(src='./bar_stack0.html', width=1200, height=600)
# Image(filename='/Users/paxton615/Github_Personal/Notes/Notes/Turtle_Trading/')



In [100]:
# mplfinance is no good in plot candles and lines

# import mplfinance as mpf

# df = spx.iloc[0:200,:]
# df['SMA'] = df.close.rolling(window=20).mean()
# df['stddev'] = df.close.rolling(window=20).std()
# df['Upper'] = df.SMA + 2* df.stddev
# df['Lower'] = df.SMA - 2* df.stddev
# df['Buy_Signal'] = np.where(df.Lower > df.close, True, False)
# df['Sell_Signal'] = np.where(df.Upper < df.close, True, False)

# tcdf = df[['Lower','Upper','SMA']]
# apd = mpf.make_addplot(tcdf)
# df.index = pd.DatetimeIndex(df['time'])
# mpf.plot(df, figratio=(8,4), type='candle', addplot=apd, volume=False, style='yahoo')
